In [1]:
from HELPpy.visualization.filecollector import FileCollector
import ipywidgets as wid
fc = FileCollector(".", title='Choose file', filter_pattern='*.csv', layout=wid.Layout(width='auto'))
display(fc)

FileCollector(path='/Users/maurizio/HELP/prova', filename='()', title='Choose file', show_hidden=False, select…

In [20]:
import os
s = ('/Users/maurizio/HELP/HELPpy/notebooks/Kidney_AE.csv', '/Users/maurizio/HELP/HELPpy/notebooks/myfilechooser.py', '/Users/maurizio/HELP/HELPpy/notebooks/Kidney_BPBeder_0.csv', '/Users/maurizio/HELP/HELPpy/notebooks/Kidney_BPBeder_10.csv', '/Users/maurizio/HELP/HELPpy/notebooks/Kidney_BIO_w_const.csv', '/Users/maurizio/HELP/HELPpy/notebooks/Kidney_BPBeder_1.csv', '/Users/maurizio/HELP/HELPpy/notebooks/prova.ipynb')
[os.path.isfile(ss) for ss in s]

[True, False, False, False, False, False, False]

In [44]:
import ipywidgets as widgets
widget = widgets.HTML(value= "a \n b" +' </p>')
display(widget)

HTML(value='a \n b </p>')

In [21]:
from tqdm.notebook import tqdm

class ipytqdm(tqdm):
    """store file metadata"""
    def __init__(self, pbar=None, **kwds):
        self.ipybar = pbar
        super().__init__(**kwds)
        if self.ipybar is not None:
            self.ipybar.max = self.total
            self.ipybar.bar_style = "danger"
    def update(self, n=1):
        super().update(n)
        if self.ipybar is not None:
            self.ipybar.value += n

 
from ipywidgets import IntProgress

pb = IntProgress(min=0, value=0)
display(pb)
with ipytqdm(total=int(1e4), desc="loading", disable=False, pbar=None) as bar:
    for i in range(int(1e4)):
        bar.update()



IntProgress(value=0)

loading:   0%|          | 0/10000 [00:00<?, ?it/s]

In [23]:
import pandas as pd  
  
import numpy as np  
  
from tqdm import tqdm  
  
tqdm.pandas(desc='pandas tqdm integration demo')

pd.read_csv("/Users/maurizio/HELP/data/CRISPRGeneEffect.csv")

    0   1   2   3   4
0  40  27  44  94  44
1  28  87  73   5  35
2  56   8  78  94  95
3  20  51  12  49  53
4  89  26  20  72  92
5  33  12  16  20  42
6  35  95  19  62  77
7  22  61  50  29  74
8  36   4  86  57  55
9  99  44  53  48  73


,Unnamed: 0,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZWILCH (55055),ZWINT (11130),ZXDA (7789),ZXDB (158586),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009)
0,ACH-000001,-0.122637,0.025881,0.034217,-0.128082,-0.031285,0.338046,-0.006439,-0.093642,0.189186,...,-0.131727,-0.039829,0.179405,0.283552,0.204513,-0.289724,-0.062972,0.074180,0.111244,-0.467908
1,ACH-000004,0.019756,-0.083640,-0.060118,-0.027417,-0.036116,-0.001056,0.312876,-0.086897,0.204434,...,-0.170329,-0.454263,0.194583,0.098989,0.126948,0.032983,-0.410392,0.113156,0.234388,-0.088306
2,ACH-000005,-0.107208,-0.023211,0.200204,0.116039,-0.172227,-0.071294,0.203270,-0.127806,-0.090981,...,-0.301695,-0.454969,-0.061959,-0.036427,0.016602,-0.201273,-0.178877,-0.055349,-0.002161,-0.186842
3,ACH-000007,-0.031027,-0.137850,0.067704,0.107988,0.007992,0.124945,0.049548,-0.220824,-0.165669,...,-0.303390,-0.507272,-0.025400,0.236659,0.072010,-0.100344,-0.462160,-0.001555,-0.325964,-0.486660
4,ACH-000009,0.008888,-0.146566,0.084471,0.089419,0.065109,0.027841,0.087943,-0.161369,0.041121,...,-0.255466,-0.288739,-0.037132,0.261444,-0.062391,-0.112703,-0.598698,0.095877,-0.026742,-0.320759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,ACH-002834,-0.055721,-0.121228,0.072790,0.021916,-0.221940,0.071135,0.134854,-0.504790,0.058356,...,-0.104959,-0.559438,-0.062461,-0.058814,-0.076221,0.024959,0.015259,-0.306562,-0.148368,-0.579576
1096,ACH-002847,-0.009973,-0.119813,0.044097,0.041358,-0.146565,0.160566,0.134247,-0.242759,-0.085035,...,-0.314751,-0.903238,0.104278,0.101270,0.026373,-0.119911,-0.289412,-0.195097,-0.206400,-0.486525
1097,ACH-002922,-0.025991,-0.007706,-0.038468,0.236576,-0.239690,0.061611,0.128461,-0.498119,0.022097,...,-0.164150,-0.743401,-0.023401,0.174912,-0.230089,-0.079342,-0.347484,-0.085302,-0.095965,-0.346272
1098,ACH-002925,-0.127639,-0.040705,0.134556,-0.047984,-0.116114,0.213144,0.198647,-0.183355,-0.004409,...,-0.275523,-0.898982,-0.050201,0.125480,-0.189114,-0.043555,-0.335270,-0.208063,-0.094741,-0.222404


In [15]:
import pandas as pd
%cd "/Users/maurizio/Google Drive/Il mio Drive/PLOS_CompBiology/4revision/data"
df = pd.read_csv("labels_Matrix_23Q4_filteredNA095_additional.csv", sep=';', index_col=0)
for fname in df.columns:
    df[[fname]].rename(columns={fname : 'label'}).to_csv(f"{fname}.csv", index=True)

/Users/maurizio/Library/CloudStorage/GoogleDrive-maurizio.giordano@icar.cnr.it/Il mio Drive/PLOS_CompBiology/4revision/data
